# **Start**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Uninstall existing scikit-learn to avoid conflicts
!pip uninstall -y scikit-learn
# Install specific versions of libraries to avoid conflicts
!pip install scikit-learn==1.5.2
!pip install bayesian-optimization
!pip install optuna
!pip install gpboost
!pip install shap
!pip install ngboost
!pip install dask[dataframe]
!pip install torch seaborn
!pip install lightgbm
!pip install xgboost
!pip install lime
!pip install interpret
!pip install optunahub
!pip install cmaes
!pip install plotly kaleido
!pip install openpyxl
!pip install -U kaleido
!pip install properscoring
!pip install XlsxWriter
!pip install cython
!pip install pgbm
!pip install torch
!pip install cp
!pip install mapie
!pip install torch skorch puncc
# Reinstall scikit-learn to the version required by ngboost
!pip uninstall -y scikit-learn
!pip install scikit-learn==1.6.1
# Reinstall numpy first
!pip install numpy==1.26.4  # Use the version compatible with catboost
# Reinstall catboost
!pip install catboost

Found existing installation: scikit-learn 1.6.1
Uninstalling scikit-learn-1.6.1:
  Successfully uninstalled scikit-learn-1.6.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 99.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 42.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 75.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.7/115.7 kB 8.6 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4030 sha256=684de70ec1251e6bf1b79c9a84026b246dd26884bae7a2d304da91d5fb5779a9
  Stored in directory: /root/.cache/pip/wheels/8b/67/f4/2caaae2146198dcb824f31a303833b07b14a5ec863fb3acd7b
Su

In [ ]:
# Restart the runtime to apply changes
import os
os._exit(00)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ngboost
from scipy.stats import randint
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.svm import SVR
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from gpboost import GPBoostRegressor
from ngboost import NGBRegressor
import optuna
import optunahub
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from interpret import show
from interpret.blackbox import LimeTabular, ShapKernel
from optuna.samplers import RandomSampler
import random
import time
from ngboost.distns import Normal
from ngboost.scores import LogScore
from scipy.stats import norm
from interpret import set_visualize_provider
from interpret.provider import InlineProvider
from interpret.glassbox import ExplainableBoostingRegressor
from interpret import show
import plotly.express as px
from io import BytesIO
from openpyxl import Workbook, load_workbook
import os
from openpyxl.drawing.image import Image as openpyxlImage
import warnings
import xlsxwriter
from openpyxl.drawing.image import Image
from pgbm.sklearn import HistGradientBoostingRegressor
import torch
from pgbm.torch import PGBM
import plotly.graph_objects as go
warnings.filterwarnings('ignore')
import pickle
import json

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
Using /root/.cache/torch_extensions/py311_cu124 as PyTorch extensions root...
Creating extension directory /root/.cache/torch_extensions/py311_cu124/split_decision...
Emitting ninja build file /root/.cache/torch_extensions/py311_cu124/split_decision/build.ninja...
Building extension module split_decision...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Loading extension module split_decision...
Using /root/.cache/torch_extensions/py311_cu124 as PyTorch extensions root...
No modifications detected for re-loaded extension module split_decision, skipping build step...
Loading extension module split_decision...


In [14]:
train_data_path = "./drive/MyDrive/Concrete_ml/train.csv"
test_data_path = "./drive/MyDrive/Concrete_ml/test.csv"
train_data = pd.read_csv(train_data_path)
test_data = pd.read_csv(test_data_path)
print("Training data loaded successfully.")
print("Test data loaded successfully.")

Training data loaded successfully.
Test data loaded successfully.


In [15]:
print("\nShape of training data:", train_data.shape)
print("First 5 rows of training data:\n", train_data.head(5))
print("\nShape of test data:", test_data.shape)
print("First 5 rows of test data:\n", test_data.head(5))


Shape of training data: (189, 8)
First 5 rows of training data:
         C    mp     FA      CA       F       W_P    Adm    str
0  280.80  70.2  858.0  1183.0    0.00  0.450000  0.610  21.56
1  372.15   0.0  975.0   525.0   52.85  0.493081  7.000  34.00
2  360.00  75.0  975.0   525.0   40.00  0.509722  7.000  28.00
3  364.30   0.0  975.0   525.0  110.40  0.503706  7.000  42.00
4  315.00  31.5  780.0  1110.0    0.00  0.370000  5.355  25.82

Shape of test data: (95, 8)
First 5 rows of test data:
         C     mp     FA     CA      F       W_P   Adm   str
0  364.30    0.0  975.0  525.0  60.40  0.503706   7.0  34.0
1  344.30   75.0  975.0  525.0  55.40  0.532965   7.0  36.0
2  390.00    0.0  975.0  525.0  60.00  0.470513   7.0  36.0
3  352.15   75.0  975.0  525.0  47.85  0.521085   7.0  35.0
4  400.00  160.0  801.0  801.0  40.00  0.300000  10.3  44.6


In [16]:
X_train = train_data.iloc[:, :-1]
y_train = train_data.iloc[:, -1]
X_test = test_data.iloc[:, :-1]
y_test = test_data.iloc[:, -1]
x_test= X_test
print("\nShape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_test:", y_test.shape)


Shape of X_train: (189, 7)
Shape of y_train: (189,)
Shape of X_test: (95, 7)
Shape of y_test: (95,)


In [17]:
# Apply z-score normalization
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Print the first five rows of the normalized data
print("\nFirst five rows of normalized X_train:")
print(X_train[:5])

print("\nFirst five rows of normalized X_test:")
print(X_test[:5])


First five rows of normalized X_train:
[[-1.56879184  0.95792014 -0.09685034  1.33138508 -0.85369002  0.16810471
  -0.08799752]
 [ 0.40300047 -0.7708921   0.73537613 -0.93459372  0.061075    0.78285857
  -0.0664621 ]
 [ 0.14074238  1.07612953  0.73537613 -0.93459372 -0.16134185  1.020321
  -0.0664621 ]
 [ 0.233558   -0.7708921   0.73537613 -0.93459372  1.05719093  0.93447436
  -0.0664621 ]
 [-0.83058388  0.00485698 -0.65166799  1.07999229 -0.85369002 -0.97347298
  -0.07200604]]

First five rows of normalized X_test:
[[ 0.233558   -0.7708921   0.73537613 -0.93459372  0.19175572  0.93447436
  -0.0664621 ]
 [-0.19814256  1.07612953  0.73537613 -0.93459372  0.1052122   1.35199213
  -0.0664621 ]
 [ 0.78829322 -0.7708921   0.73537613 -0.93459372  0.18483223  0.4608195
  -0.0664621 ]
 [-0.02870009  1.07612953  0.73537613 -0.93459372 -0.02546852  1.18246784
  -0.0664621 ]
 [ 1.0041435   3.1694207  -0.50229401  0.01587763 -0.16134185 -1.97235346
  -0.05534052]]


In [18]:
# Define the model classes
model_classes = {
    'Random Forest': RandomForestRegressor,
    'Gradient Boosting': GradientBoostingRegressor,
    'XGBoost': XGBRegressor,
    'LightGBM': LGBMRegressor,
    'GPBoost': GPBoostRegressor,
    'CatBoost': CatBoostRegressor,
    'NGBoost': NGBRegressor
}

feature_names = ['C', 'mp', 'FA', 'CA', 'F', 'W_P', 'Adm']


In [8]:
def plot_best_scores(best_scores_ran, excel_file_path):
    # Extract the best pruner for each model based on RMSE and correlation coefficient
    best_rmse_scores = {}
    best_corr_coef_scores = {}

    for (model_name, pruner_name), scores in best_scores_ran.items():
        # Initialize if not already present
        if model_name not in best_rmse_scores:
            best_rmse_scores[model_name] = (scores['test_rmse'], pruner_name)
        if model_name not in best_corr_coef_scores:
            best_corr_coef_scores[model_name] = (scores['test_corr_coef'], pruner_name)

        # Update if better scores are found
        if scores['test_rmse'] < best_rmse_scores[model_name][0]:
            best_rmse_scores[model_name] = (scores['test_rmse'], pruner_name)
        if scores['test_corr_coef'] > best_corr_coef_scores[model_name][0]:
            best_corr_coef_scores[model_name] = (scores['test_corr_coef'], pruner_name)

    # Prepare data for plotting
    model_names_rmse = [f"{model} ({pruner})" for model, (rmse, pruner) in best_rmse_scores.items()]
    rmse_values = [rmse for rmse, _ in best_rmse_scores.values()]

    model_names_corr = [f"{model} ({pruner})" for model, (corr, pruner) in best_corr_coef_scores.items()]
    corr_values = [corr for corr, _ in best_corr_coef_scores.values()]

    # Plot RMSE
    plt.figure(figsize=(12, 6))
    bars_rmse = plt.bar(model_names_rmse, rmse_values, color='skyblue')

    # Highlight the best model
    best_rmse_index = np.argmin(rmse_values)
    bars_rmse[best_rmse_index].set_color('orange')

    # Annotate the bars with the RMSE scores
    for i, bar in enumerate(bars_rmse):
        plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height() - 0.05,
                 f'{rmse_values[i]:.5f}', ha='center', va='bottom', color='black')

    # Add labels and title for the RMSE bar chart
    plt.xlabel('Model (Pruner)')
    plt.ylabel('Test RMSE')
    plt.title('Best Test RMSE for Each Model')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    rmse_image_path = 'rmse_plot.png'
    plt.savefig(rmse_image_path)
    plt.close()

    # Plot Correlation Coefficient
    plt.figure(figsize=(12, 6))
    bars_corr = plt.bar(model_names_corr, corr_values, color='lightgreen')

    # Highlight the best model
    best_corr_index = np.argmax(corr_values)
    bars_corr[best_corr_index].set_color('orange')

    # Annotate the bars with the correlation coefficient scores
    for i, bar in enumerate(bars_corr):
        plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height() - 0.05,
                 f'{corr_values[i]:.5f}', ha='center', va='bottom', color='black')

    # Add labels and title for the correlation coefficient bar chart
    plt.xlabel('Model (Pruner)')
    plt.ylabel('Correlation Coefficient')
    plt.title('Best Correlation Coefficient for Each Model')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    corr_image_path = 'corr_plot.png'
    plt.savefig(corr_image_path)
    plt.close()

    # Load the existing Excel file
    workbook = load_workbook(excel_file_path)

    # Create a new sheet for the plots
    sheet_name = 'Best Models Plots'
    if sheet_name in workbook.sheetnames:
        sheet = workbook[sheet_name]
    else:
        sheet = workbook.create_sheet(title=sheet_name)

    # Insert images into the new Excel sheet
    img_rmse = Image(rmse_image_path)
    img_corr = Image(corr_image_path)

    # Insert images
    sheet.add_image(img_rmse, 'A1')
    sheet.add_image(img_corr, 'A20')  # Adjust the position as needed

    # Save the workbook
    workbook.save(excel_file_path)

    # Clean up the image files
    os.remove(rmse_image_path)
    os.remove(corr_image_path)

# Example usage
# plot_best_scores(best_scores_ran, 'path_to_your_excel_file.xlsx')

In [9]:
def generate_interpretml_explanations_summary_pruners(
    results_dict, X_train, y_train, X_test, feature_names, instance_indices=None, excel_file_path=None
):
    if instance_indices is None:
        instance_indices = range(len(X_test))
    elif isinstance(instance_indices, int):
        instance_indices = [instance_indices]

    valid_indices = [idx for idx in instance_indices if 0 <= idx < len(X_test)]
    if not valid_indices:
        print("No valid instance indices provided.")
        return

    if isinstance(X_test, pd.DataFrame):
        instances_to_explain = X_test.iloc[valid_indices]
    else:
        instances_to_explain = X_test[valid_indices]

    best_model_pruners = {}
    for model_key, model_info in results_dict.items():
        if isinstance(model_key, tuple):
            model_name, pruner_name = model_key
        else:
            model_name = model_key
            pruner_name = None

        best_score = model_info.get('best_score')
        if best_score is None:
            print(f"No 'best_score' found for {model_key}. Skipping this combination.")
            continue

        if model_name not in best_model_pruners:
            best_model_pruners[model_name] = {
                'pruner_name': pruner_name,
                'model_info': model_info,
                'best_score': best_score
            }
        else:
            current_best_score = best_model_pruners[model_name]['best_score']
            if best_score < current_best_score:
                best_model_pruners[model_name] = {
                    'pruner_name': pruner_name,
                    'model_info': model_info,
                    'best_score': best_score
                }

    for model_name, info in best_model_pruners.items():
        pruner_name = info['pruner_name']
        model_info = info['model_info']
        best_params = model_info['best_params']
        model_class = model_classes.get(model_name)

        if model_class is None:
            print(f"Model {model_name} is not supported or not available.")
            continue

        if model_name == 'CatBoost':
            best_params['verbose'] = 0

        model = model_class(**best_params)
        model.fit(X_train, y_train)

        def predict_fn(data):
            return model.predict(data)

        if isinstance(X_train, pd.DataFrame):
            data_for_explainer = X_train.values
        else:
            data_for_explainer = X_train

        if isinstance(instances_to_explain, pd.DataFrame):
            data_for_explanation = instances_to_explain.values
        else:
            data_for_explanation = instances_to_explain

        # Generate LIME explanations
        lime_explainer = LimeTabular(
            predict_fn,
            data=data_for_explainer,
            feature_names=feature_names,
            random_state=1,
            mode='regression'
        )
        lime_explanation = lime_explainer.explain_local(data_for_explanation)

        feature_importances_lime = {}
        num_instances = len(valid_indices)

        for idx in range(num_instances):
            explanation = lime_explanation.data(idx)
            for feature_name, feature_score in zip(explanation['names'], explanation['scores']):
                feature_importances_lime[feature_name] = feature_importances_lime.get(feature_name, 0) + abs(feature_score)

        feature_importances_lime = {k: v / num_instances for k, v in feature_importances_lime.items()}
        feature_importances_lime = {k: round(v, 3) for k, v in feature_importances_lime.items()}

        # Generate SHAP explanations using ShapKernel
        try:
            shap_explainer = ShapKernel(predict_fn, data_for_explainer, feature_names=feature_names)
            shap_explanation = shap_explainer.explain_local(data_for_explanation)

            feature_importances_shap = {}
            for idx in range(num_instances):
                explanation = shap_explanation.data(idx)
                for feature_name, feature_score in zip(explanation['names'], explanation['scores']):
                    feature_importances_shap[feature_name] = feature_importances_shap.get(feature_name, 0) + abs(feature_score)

            feature_importances_shap = {k: v / num_instances for k, v in feature_importances_shap.items()}
            feature_importances_shap = {k: round(v, 3) for k, v in feature_importances_shap.items()}
        except Exception as e:
            print(f"Could not compute SHAP values for model {model_name}: {e}")
            continue

        # Plot LIME and SHAP feature importances side by side
        fig, axes = plt.subplots(1, 2, figsize=(34, 36))

        # Plot LIME feature importances
        lime_importances_df = pd.DataFrame.from_dict(
            feature_importances_lime, orient='index', columns=['importance']
        )
        lime_importances_df.sort_values(by='importance', ascending=False, inplace=True)
        lime_importances_df.plot(kind='bar', legend=False, color='skyblue', ax=axes[0])
        axes[0].set_title(f"LIME Feature Importances for {model_name}")
        axes[0].set_ylabel("Average Absolute Importance Score")
        axes[0].set_xlabel("Features")
        axes[0].tick_params(axis='x', rotation=45)

        for p in axes[0].patches:
            height = p.get_height()
            axes[0].annotate(f'{height:.3f}',
                             (p.get_x() + p.get_width() / 2., height),
                             ha='center', va='bottom', fontsize=8)

        # Plot SHAP feature importances
        shap_importances_df = pd.DataFrame.from_dict(
            feature_importances_shap, orient='index', columns=['importance']
        )
        shap_importances_df.sort_values(by='importance', ascending=False, inplace=True)
        shap_importances_df.plot(kind='bar', legend=False, color='orange', ax=axes[1])
        axes[1].set_title(f"SHAP Feature Importances for {model_name}")
        axes[1].set_ylabel("Average Absolute SHAP Value")
        axes[1].set_xlabel("Features")
        axes[1].tick_params(axis='x', rotation=45)

        for p in axes[1].patches:
            height = p.get_height()
            axes[1].annotate(f'{height:.3f}',
                             (p.get_x() + p.get_width() / 2., height),
                             ha='center', va='bottom', fontsize=8)

        plt.tight_layout()

        # Save plots as images
        image_path = f'feature_importances_{model_name}.png'
        fig.savefig(image_path)
        plt.close(fig)

        # Optionally insert images and scores into an Excel file
        if excel_file_path:
            workbook = load_workbook(excel_file_path)
            sheet_name = f'{model_name} Explanations'
            if sheet_name in workbook.sheetnames:
                sheet = workbook[sheet_name]
            else:
                sheet = workbook.create_sheet(title=sheet_name)

            # Insert images into the new Excel sheet
            img = Image(image_path)
            sheet.add_image(img, 'A1')

            # Create a new sheet for feature importance scores
            scores_sheet_name = f'{model_name} Scores'
            if scores_sheet_name in workbook.sheetnames:
                scores_sheet = workbook[scores_sheet_name]
            else:
                scores_sheet = workbook.create_sheet(title=scores_sheet_name)

            # Write LIME scores
            scores_sheet.append(['Feature', 'LIME Importance'])
            for feature, importance in feature_importances_lime.items():
                scores_sheet.append([feature, importance])

            # Write SHAP scores
            scores_sheet.append(['Feature', 'SHAP Importance'])
            for feature, importance in feature_importances_shap.items():
                scores_sheet.append([feature, importance])

            # Save the workbook
            workbook.save(excel_file_path)

            # Clean up the image file
            os.remove(image_path)

# **Autosampler by Optuna**

In [10]:
# Suppress Optuna's verbose output
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Define objective and metric functions for PGBM
def mseloss_objective(yhat, y, sample_weight=None):
    if not torch.is_tensor(yhat):
        yhat = torch.from_numpy(np.array(yhat)).float()
    if not torch.is_tensor(y):
        y = torch.from_numpy(np.array(y)).float()
    gradient = yhat - y
    hessian = torch.ones_like(yhat)
    return gradient, hessian

def rmseloss_metric(yhat, y, sample_weight=None):
    if not torch.is_tensor(yhat):
        yhat = torch.from_numpy(np.array(yhat)).float()
    if not torch.is_tensor(y):
        y = torch.from_numpy(np.array(y)).float()
    loss = torch.sqrt(torch.mean((yhat - y) ** 2))
    return loss

def hyperparameter_tuning_all(X_train, y_train, X_test, y_test, excel_path):
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    X_test = np.array(X_test)
    y_test = np.array(y_test)

    models = {
        'Random Forest': (RandomForestRegressor, {
            'n_estimators': [100, 200, 300, 500, 700],
            'criterion': ['squared_error', 'absolute_error', 'friedman_mse', 'poisson'],
            'max_depth': [None, 10, 20, 30, 40],
            'min_samples_split': [2, 5, 10, 0.01],
            'min_samples_leaf': [1, 3, 5, 0.01],
            'min_weight_fraction_leaf': [0.0, 0.01, 0.1, 0.2],
            'max_features': [1.0, 'sqrt', 'log2', 0.3, 0.5],
            'max_leaf_nodes': [None, 50, 100, 200],
            'min_impurity_decrease': [0.0, 0.01, 0.1, 0.2],
            'n_jobs': [-1],
            'random_state': [42],
            'verbose': [0],
            'warm_start': [False],
            'ccp_alpha': [0.0, 0.001, 0.01, 0.05, 0.1]
        }),
        'Gradient Boosting': (GradientBoostingRegressor, {
            'loss': ['squared_error', 'absolute_error', 'huber', 'quantile'],
            'learning_rate': [0.01, 0.05, 0.1, 0.2],
            'n_estimators': [100, 200, 300, 500, 700],
            'subsample': [1.0, 0.9, 0.7, 0.5],
            'criterion': ['friedman_mse', 'squared_error'],
            'min_samples_split': [2, 5, 10, 0.01],
            'min_samples_leaf': [1, 3, 5, 0.01],
            'min_weight_fraction_leaf': [0.0, 0.01, 0.05, 0.1],
            'max_depth': [3, 5, 7, 10],
            'min_impurity_decrease': [0.0, 0.01, 0.1],
            'init': [None],
            'random_state': [42],
            'max_features': [None, 'sqrt', 'log2', 0.5],
            'alpha': [0.9, 0.5, 0.1],
            'verbose': [0],
            'max_leaf_nodes': [None, 10, 30, 50],
            'warm_start': [False],
            'validation_fraction': [0.1],
            'n_iter_no_change': [None, 10, 20],
            'tol': [1e-4, 1e-3],
            'ccp_alpha': [0.0, 0.001, 0.01]
        }),
        'XGBoost': (XGBRegressor, {
            'n_estimators': [100, 200, 300, 400, 500],
            'learning_rate': [0.01, 0.05, 0.1, 0.15],
            'max_depth': [3, 5, 7],
            'min_child_weight': [1, 3, 5],
            'gamma': [0, 0.1, 0.5, 1],
            'subsample': [0.5, 0.6, 0.7, 0.8, 0.9],
            'colsample_bytree': [0.5, 0.7, 0.9],
            'colsample_bylevel': [0.5, 0.7, 0.9],
            'reg_alpha': [0, 0.01, 0.1, 1],
            'reg_lambda': [0.1, 1, 5, 10],
            'objective': ['reg:squarederror'],
            'random_state': [42],
            'n_jobs': [-1]
        }),
        'LightGBM': (LGBMRegressor, {
            'n_estimators': [100, 200, 300, 400, 500],
            'learning_rate': [0.01, 0.05, 0.1, 0.15],
            'num_leaves': [15, 31, 63],
            'max_depth': [3, 5, 7, -1],
            'min_child_samples': [1, 5, 10, 20],
            'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
            'colsample_bytree': [0.5, 0.7, 0.9, 1.0],
            'reg_alpha': [0, 0.01, 0.1, 1],
            'reg_lambda': [0, 0.1, 1, 10],
            'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1],
            'bagging_freq': [0, 1, 5],
            'objective': ['regression'],
            'random_state': [42],
            'n_jobs': [-1],
            'verbose': [-1]
        }),
        'GPBoost': (GPBoostRegressor, {
            'n_estimators': [100, 200, 300, 400, 500],
            'learning_rate': [0.01, 0.05, 0.1, 0.15],
            'max_depth': [3, 5, 7, -1],
            'num_leaves': [15, 31, 63],
            'min_child_samples': [1, 5, 10, 20],
            'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
            'colsample_bytree': [0.5, 0.7, 0.9, 1.0],
            'reg_alpha': [0, 0.1, 0.5, 1.0],
            'reg_lambda': [0, 0.1, 0.5, 1.0],
            'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1],
            'random_state': [42],
            'n_jobs': [-1],
            'verbose': [-1]
        }),
        'CatBoost': (CatBoostRegressor, {
            'iterations': [200, 500, 1000],
            'learning_rate': [0.01, 0.03, 0.05, 0.1],
            'depth': [4, 6, 8, 10],
            'l2_leaf_reg': [1, 3, 5, 7, 9],
            'border_count': [32, 64, 128],
            'min_data_in_leaf': [1, 5, 10, 20],
            'rsm': [0.6, 0.8, 1.0],
            'bagging_temperature': [0, 1, 10],
            'random_seed': [42],
            'verbose': [0]
        }),
        'NGBoost': (NGBRegressor, {
            'n_estimators': [200, 500, 1000],
            'learning_rate': [0.01, 0.03, 0.05, 0.1],
            'natural_gradient': [True, False],
            'minibatch_frac': [0.5, 0.7, 0.9, 1.0],
            'col_sample': [0.5, 0.7, 0.9, 1.0],
            'Dist': [Normal],
            'Score': [LogScore],
            'random_state': [42],
            'verbose': [0]
        }),
        'HistGradientBoosting': (HistGradientBoostingRegressor, {
            'learning_rate': [0.01, 0.05, 0.1, 0.15],
            'max_iter': [100, 200, 300, 400, 500],
            'max_depth': [3, 5, 7, None],
            'min_samples_leaf': [5, 10, 20],
            'max_leaf_nodes': [15, 31, 63, None],
            'l2_regularization': [0.0, 0.1, 0.5, 1.0],
            'max_bins': [64, 128, 255],
            'early_stopping': [True, False],
            'validation_fraction': [0.1, 0.2],
            'n_iter_no_change': [5, 10, 15],
            'loss': ['squared_error'],
            'random_state': [42],
            'verbose': [0]
        }),
        'PGBM': (PGBM, {}) # PGBM is handled separately
    }


    pruners = [
        optuna.pruners.MedianPruner(),
        optuna.pruners.NopPruner(),
        optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=3),
        optuna.pruners.PercentilePruner(25.0),
        optuna.pruners.SuccessiveHalvingPruner(),
        optuna.pruners.HyperbandPruner(),
        optuna.pruners.ThresholdPruner(lower=0.1),
        optuna.pruners.WilcoxonPruner()
    ]

    best_scores = {}
    predictions_df = pd.DataFrame({'Actual': y_test})

    for model_name, (model_class, param_space) in models.items():
        for pruner in pruners:
            pruner_name = pruner.__class__.__name__
            print(f"Running Optuna for {model_name} with {pruner_name}...")
            try:
                sampler = optunahub.load_module("samplers/auto_sampler").AutoSampler()
                study = optuna.create_study(direction='minimize', sampler=sampler, pruner=pruner)

                if model_name == 'PGBM':
                    def pgbm_objective(trial):
                        params = {
                            'n_estimators': trial.suggest_categorical('n_estimators', [100, 200, 300, 500]),
                            'learning_rate': trial.suggest_categorical('learning_rate', [0.01, 0.05, 0.1, 0.15]),
                            'max_leaves': trial.suggest_int('max_leaves', 15, 63),
                            'min_split_gain': trial.suggest_categorical('min_split_gain', [0.0, 0.1, 0.5, 1.0]),
                            'reg_lambda': trial.suggest_categorical('reg_lambda', [0.1, 1.0, 5.0, 10.0]),
                            'feature_fraction': trial.suggest_categorical('feature_fraction', [0.5, 0.7, 0.9, 1.0]),
                            'bagging_fraction': trial.suggest_categorical('bagging_fraction', [0.5, 0.7, 0.9, 1.0]),
                            'tree_correlation': trial.suggest_categorical('tree_correlation', [0.0, 0.1, 0.2, 0.3]),
                            'min_data_in_leaf': trial.suggest_categorical('min_data_in_leaf', [3, 5, 10, 20]),
                            'max_bin': trial.suggest_categorical('max_bin', [64, 128, 256]),
                            'distribution': trial.suggest_categorical('distribution', ['normal', 'studentt', 'laplace']),
                            'objective': 'mse',
                            'metric': 'rmse',
                            'random_state': 42,
                            'verbose': 0
                        }

                        model = PGBM()
                        model.train((X_train, y_train), objective=mseloss_objective, metric=rmseloss_metric, params=params)
                        y_pred = model.predict(X_test)
                        mse = mean_squared_error(y_test, y_pred)
                        return mse

                    study.optimize(pgbm_objective, n_trials=50)
                else:
                    def objective(trial, model_name, model_class, param_space):
                        params = {}
                        for key, values in param_space.items():
                            if isinstance(values, list):
                                params[key] = trial.suggest_categorical(key, values)
                            elif isinstance(values, tuple):
                                if len(values) == 2:
                                    params[key] = trial.suggest_float(key, values[0], values[1])
                                elif len(values) == 3 and isinstance(values[2], bool) and values[2]:
                                    params[key] = trial.suggest_int(key, values[0], values[1])
                                else:
                                    raise ValueError(f"Invalid parameter range for {key}")

                        model = model_class(**params)
                        model.fit(X_train, y_train)
                        y_pred = model.predict(X_test)
                        mse = mean_squared_error(y_test, y_pred)
                        return mse

                    study.optimize(lambda trial: objective(trial, model_name, model_class, param_space), n_trials=50)

                best_params = study.best_params
                best_model = model_class(**best_params) if model_name != 'PGBM' else PGBM()
                if model_name == 'PGBM':
                    best_model.train((X_train, y_train), objective=mseloss_objective, metric=rmseloss_metric, params=best_params)
                else:
                    best_model.fit(X_train, y_train)
                y_pred = best_model.predict(X_test)

                mse = mean_squared_error(y_test, y_pred)
                rmse = np.sqrt(mse)
                corr_coef = np.corrcoef(y_test, y_pred)[0, 1]

                # Save predictions to DataFrame
                predictions_df[f'{model_name}_{pruner_name}_Predicted'] = y_pred

                best_scores[(model_name, pruner_name)] = {
                    'best_score': mse,
                    'best_params': best_params,
                    'test_mse': mse,
                    'test_rmse': rmse,
                    'test_corr_coef': corr_coef,
                    'pruner': pruner_name
                }
                print(f"Best MSE for {model_name} with {pruner_name}: {mse} with params: {best_params}")
                print(f"Best RMSE for {model_name} with {pruner_name}: {rmse}")
                print(f"Correlation Coefficient for {model_name} with {pruner_name}: {corr_coef}")
            except Exception as e:
                print(f"Failed to run Optuna for {model_name} with {pruner_name}. Error: {e}")

    # Write predictions to Excel
    with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a') as writer:
        predictions_df.to_excel(writer, sheet_name='Predictions', index=False)

    if best_scores:
        best_model_name, best_pruner_name = min(best_scores, key=lambda k: best_scores[k]['test_mse'])
        best_model_info = best_scores[(best_model_name, best_pruner_name)]
        print(f"\nBest model on test data: {best_model_name} with {best_pruner_name}")
        print(f"Test MSE: {best_model_info['test_mse']}")
        print(f"Test RMSE: {best_model_info['test_rmse']}")
        print(f"Correlation Coefficient: {best_model_info['test_corr_coef']}")
        print(f"Best Parameters: {best_model_info['best_params']}")
        print(f"Pruner Used: {best_model_info['pruner']}")
    else:
        print("No valid model configurations found.")

    return best_scores

# Example usage
best_scores_autosampler = hyperparameter_tuning_all(X_train, y_train, X_test, y_test, "./drive/MyDrive/Concrete_ml/test.xlsx")

Running Optuna for Random Forest with MedianPruner...
Best MSE for Random Forest with MedianPruner: 11.111950168679973 with params: {'n_estimators': 300, 'criterion': 'friedman_mse', 'max_depth': 10, 'min_samples_split': 0.01, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0.0, 'max_features': 0.5, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.1, 'n_jobs': -1, 'random_state': 42, 'verbose': 0, 'warm_start': False, 'ccp_alpha': 0.05}
Best RMSE for Random Forest with MedianPruner: 3.333459189592693
Correlation Coefficient for Random Forest with MedianPruner: 0.9447661531955767
Running Optuna for Random Forest with NopPruner...
Best MSE for Random Forest with NopPruner: 11.767782679686066 with params: {'n_estimators': 500, 'criterion': 'friedman_mse', 'max_depth': 10, 'min_samples_split': 2, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0.0, 'max_features': 0.3, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'n_jobs': -1, 'random_state': 42, 'verbose': 0, 'warm_start'

In [24]:
best_scores_autosampler

{('Random Forest', 'MedianPruner'): {'best_score': 11.111950168679973,
  'best_params': {'n_estimators': 300,
   'criterion': 'friedman_mse',
   'max_depth': 10,
   'min_samples_split': 0.01,
   'min_samples_leaf': 1,
   'min_weight_fraction_leaf': 0.0,
   'max_features': 0.5,
   'max_leaf_nodes': None,
   'min_impurity_decrease': 0.1,
   'n_jobs': -1,
   'random_state': 42,
   'verbose': 0,
   'warm_start': False,
   'ccp_alpha': 0.05},
  'test_mse': 11.111950168679973,
  'test_rmse': 3.333459189592693,
  'test_corr_coef': 0.9447661531955767,
  'pruner': 'MedianPruner'},
 ('Random Forest', 'NopPruner'): {'best_score': 11.767782679686066,
  'best_params': {'n_estimators': 500,
   'criterion': 'friedman_mse',
   'max_depth': 10,
   'min_samples_split': 2,
   'min_samples_leaf': 1,
   'min_weight_fraction_leaf': 0.0,
   'max_features': 0.3,
   'max_leaf_nodes': None,
   'min_impurity_decrease': 0.0,
   'n_jobs': -1,
   'random_state': 42,
   'verbose': 0,
   'warm_start': False,
   'ccp_

# **Best Model Analysis**

In [20]:
def get_best_models_and_predict(best_scores_autosampler, X_train, y_train, X_test, y_test, file_path):
    # Convert input data to NumPy arrays
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    X_test = np.array(X_test)
    y_test = np.array(y_test)

    # Mapping for model creation based on dictionary keys
    model_mapping = {
        'Random Forest': RandomForestRegressor,
        'Gradient Boosting': GradientBoostingRegressor,
        'XGBoost': XGBRegressor,
        'LightGBM': LGBMRegressor,
        'CatBoost': CatBoostRegressor,
        'GPBoost': GPBoostRegressor,
        'NGBoost': NGBRegressor,
        'HistGradientBoosting': HistGradientBoostingRegressor,
        'PGBM': PGBM  # PGBM is handled separately
    }

    # Dictionary to store the best model for each type
    best_models = {}

    # Iterate over the dictionary to find the best pruner for each model type
    for (model_name, pruner), params in best_scores_autosampler.items():
        current_score = params.get('test_mse', np.inf)
        if model_name not in best_models or current_score < best_models[model_name]['score']:
            best_models[model_name] = {
                'score': current_score,
                'params': params['best_params'],
                'pruner': pruner
            }

    # Prepare a DataFrame to store predictions
    df = pd.read_csv(file_path)

    # Iterate over the best models to train and predict
    for model_name, model_info in best_models.items():
        best_params = model_info['params']
        model_class = model_mapping.get(model_name)

        if model_class is None:
            print(f"Model {model_name} is not supported or not available.")
            continue

        # Handle specific parameters or settings for model if needed
        if model_name == 'CatBoost':
            best_params.pop('verbose', None)  # Remove 'verbose' for CatBoost

        # Create an instance of the best model with the best parameters
        if model_name == 'PGBM':
            model = model_class()
            model.train((X_train, y_train), objective=mseloss_objective, metric=rmseloss_metric, params=best_params)
            predictions = model.predict(X_test)
        else:
            model = model_class(**best_params)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)

        # Add predictions to the DataFrame
        df[f'{model_name} Predictions'] = predictions

        # Plot actual vs. predicted
        plt.figure(figsize=(10, 6))
        plt.scatter(y_test, predictions, alpha=0.6)
        plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', color='red', lw=2)
        plt.xlabel("Actual")
        plt.ylabel("Predicted")
        plt.title(f"Actual vs. Predicted Values ({model_name})")
        plt.grid(True)
        plt.tight_layout()

        # Save the plot temporarily
        plot_path = f'temp_plot_{model_name}.png'
        plt.savefig(plot_path)
        plt.close()

    # Save predictions and plots to Excel
    with pd.ExcelWriter(file_path.replace('.csv', '_results.xlsx'), engine='xlsxwriter') as writer:
        # Write data to Excel
        df.to_excel(writer, sheet_name='Data', index=False)

        # Get the xlsxwriter objects
        workbook = writer.book

        # Insert each plot into a separate worksheet
        for model_name in best_models.keys():
            # Shorten the worksheet name to fit within Excel's 31 character limit
            short_model_name = ''.join([word[0] for word in model_name.split()])  # e.g., 'RF' for 'Random Forest'
            sheet_name = f'{short_model_name}_Plot'

            worksheet = workbook.add_worksheet(sheet_name)
            writer.sheets[sheet_name] = worksheet
            plot_path = f'temp_plot_{model_name}.png'
            worksheet.insert_image('A1', plot_path)

    # Clean up temporary plot files
    for model_name in best_models.keys():
        os.remove(f'temp_plot_{model_name}.png')

    return df, best_models

# Call the function
df, best_models = get_best_models_and_predict(best_scores_autosampler, X_train, y_train, X_test, y_test, "./drive/MyDrive/Concrete_ml/test.csv")

0:	learn: 9.4531304	total: 3.18ms	remaining: 1.59s
1:	learn: 8.9175711	total: 3.64ms	remaining: 907ms
2:	learn: 8.5164373	total: 4ms	remaining: 662ms
3:	learn: 8.0577320	total: 4.33ms	remaining: 537ms
4:	learn: 7.6843900	total: 4.68ms	remaining: 463ms
5:	learn: 7.3505662	total: 5.02ms	remaining: 413ms
6:	learn: 7.1992299	total: 5.86ms	remaining: 413ms
7:	learn: 6.9389102	total: 6.24ms	remaining: 383ms
8:	learn: 6.7106062	total: 6.57ms	remaining: 358ms
9:	learn: 6.4791081	total: 6.92ms	remaining: 339ms
10:	learn: 6.2549667	total: 7.24ms	remaining: 322ms
11:	learn: 6.0612454	total: 7.57ms	remaining: 308ms
12:	learn: 5.9139244	total: 7.86ms	remaining: 295ms
13:	learn: 5.8046625	total: 8.17ms	remaining: 284ms
14:	learn: 5.6674447	total: 8.53ms	remaining: 276ms
15:	learn: 5.5520458	total: 10ms	remaining: 303ms
16:	learn: 5.4099102	total: 10.4ms	remaining: 296ms
17:	learn: 5.3085295	total: 10.7ms	remaining: 287ms
18:	learn: 5.1853627	total: 11ms	remaining: 279ms
19:	learn: 5.1005407	total: 1

In [21]:
plot_best_scores(best_scores_autosampler,"./drive/MyDrive/Concrete_ml/test_results.xlsx")

In [23]:
generate_interpretml_explanations_summary_pruners(best_scores_autosampler, X_train, y_train, x_test, feature_names,excel_file_path = "./drive/MyDrive/Concrete_ml/test_results.xlsx")

  0%|          | 0/95 [00:00<?, ?it/s]

  0%|          | 0/95 [00:00<?, ?it/s]

  0%|          | 0/95 [00:00<?, ?it/s]

  0%|          | 0/95 [00:00<?, ?it/s]

  0%|          | 0/95 [00:00<?, ?it/s]

  0%|          | 0/95 [00:00<?, ?it/s]

  0%|          | 0/95 [00:00<?, ?it/s]

Model HistGradientBoosting is not supported or not available.
Model PGBM is not supported or not available.
